<center><b><font size=6>Text data pre-processing</font></b></center>

In this exercice, we shall load a database of email messages and pre-format them so that we can design automated classification methods or use off-the-shelf classifiers.

"What is there to pre-process?" you might ask. Well, actually, text data comes in a very noisy form that we, humans, have become accustomed to and filter out effortlessly to grasp the core meaning of the text. It has a lot of formatting (fonts, colors, typography...), punctuation, abbreviations, common words, grammatical rules, etc. that we might wish to discard before even starting the data analysis.

Here are some pre-processing steps that can be performed on text:
1. loading the data, removing attachements, merging title and body;
2. tokenizing - splitting the text into atomic "words";
3. removal of stop-words - very common words;
4. removal of non-words - punctuation, numbers, gibberish;
3. lemmatization - merge together "find", "finds", "finder".

The final goal is to be able to represent a document as a mathematical object, e.g. a vector, that our machine learning black boxes can process.

# Load the data

Let's first load the emails.

In [1]:
import os
data_switch=1
if(data_switch==0):
    train_dir = '../data/ling-spam/train-mails/'
    email_path = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]
else:
    train_dir = '../data/lingspam_public/bare/'
    email_path = []
    email_label = []
    for d in os.listdir(train_dir):
        folder = os.path.join(train_dir,d)
        email_path += [os.path.join(folder,f) for f in os.listdir(folder)]
        email_label += [f[0:3]=='spm' for f in os.listdir(folder)]
print("number of emails",len(email_path))
email_nb = 0 # try 8 for a spam example
print("email file:", email_path[email_nb])
print("email is a spam:", email_label[email_nb])
print(open(email_path[email_nb]).read())

number of emails 2893
email file: ../data/lingspam_public/bare/part10/9-902msg1.txt
email is a spam: False
Subject: new books : generative studies

we would like to bring to your attention to two new publications from john benjamins publishing in the field of generative studies : minimal words in a minimal syntax . word formation in swedish . gunlg josefsson 1998 ix , 199 pp . lingvistic aktuell / linguistics today , 19 us / canada : cloth : 1 55619 903 1 price : us $ 75 . 00 rest of the world : cloth : 90 272 2740 3 price : nlg 150 john benjamins publishing web site : http : / / www . benjamins . com for further information via e-mail : service @ benjamins . com in minimal words in a minimal syntax the author combines a detailed description of the morphological structure of words in swedish with a new approach to theoretical morphology based on the minimalist program of chomsky ( 1995 ) ( as developed for syntactic structure ) . the x - bar theoretic approach to word structure of the 

# Filtering out the noise

One nice thing about scikit-learn is that is has lots of preprocessing utilities. Like [`CountVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) for instance, that converts a collection of text documents to a matrix of token counts.

- To remove stop-words, we set: `stop_words='english'`
- To convert all words to lowercase: `lowercase=True`
- The default tokenizer in scikit-learn removes punctuation and only keeps words of more than 2 letters.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
countvect = CountVectorizer(input='filename', stop_words='english', lowercase=True)
word_count = countvect.fit_transform(email_path)

In [3]:
print("Number of documents:", len(email_path))
words = countvect.get_feature_names()
print("Number of words:", len(words))
print("Document - words matrix:", word_count.shape)
print("First words:", words[0:100])

Number of documents: 2893
Number of words: 60618
Document - words matrix: (2893, 60618)
First words: ['00', '000', '0000', '00001', '00003000140', '00003003958', '00007', '0001', '00010', '00014', '0003', '00036', '000bp', '000s', '000yen', '001', '0010', '0010010034', '0011', '00133', '0014', '00170', '0019', '00198', '002', '002656', '0027', '003', '0030', '0031', '00333', '0037', '0039', '003n7', '004', '0041', '0044', '0049', '005', '0057', '006', '0067', '007', '00710', '0073', '0074', '00799', '008', '009', '00919680', '0094', '00a', '00am', '00arrival', '00b', '00coffee', '00congress', '00d', '00dinner', '00f', '00h', '00hfstahlke', '00i', '00j', '00l', '00m', '00p', '00pm', '00r', '00t', '00tea', '00the', '00uzheb', '01', '0100', '01003', '01006', '0104', '0106', '01075', '0108', '011', '0111', '0117', '0118', '01202', '01222', '01223', '01225', '01232', '01235', '01273', '013', '0131', '01334', '0135', '01364', '0139', '013953', '013a']


# Even better filtering

That's already quite ok, but this pre-processing does not perform lemmatization, the list of stop-words could be better and we could wish to remove non-english words (misspelled, with numbers, etc.).

A slightly better preprocessing uses the [Natural Language Toolkit](https://www.nltk.org/https://www.nltk.org/). The one below:
- tokenizes;
- removes punctuation;
- removes stop-words;
- removes non-English and misspelled words (optional);
- removes 1-character words;
- removes non-alphabetical words (numbers and codes essentially).

In [4]:
# If this is the first time you run this notebook, run the lines below
import nltk
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/emmanuel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/emmanuel/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/emmanuel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
from nltk import wordpunct_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import words
from string import punctuation
class LemmaTokenizer(object):
    def __init__(self, remove_non_words=True):
        self.wnl = WordNetLemmatizer()
        self.stopwords = set(stopwords.words('english'))
        self.words = set(words.words())
        self.remove_non_words = remove_non_words
    def __call__(self, doc):
        # tokenize words and punctuation
        word_list = wordpunct_tokenize(doc)
        # remove stopwords
        word_list = [word for word in word_list if word not in self.stopwords]
        # remove non words
        if(self.remove_non_words):
            word_list = [word for word in word_list if word in self.words]
        # remove 1-character words
        word_list = [word for word in word_list if len(word)>1]
        # remove non alpha
        word_list = [word for word in word_list if word.isalpha()]
        return [self.wnl.lemmatize(t) for t in word_list]

countvect = CountVectorizer(input='filename',tokenizer=LemmaTokenizer(remove_non_words=True))
word_count = countvect.fit_transform(email_path)
feat2word = {v: k for k, v in countvect.vocabulary_.items()}

In [6]:
print("Number of documents:", len(email_path))
words = countvect.get_feature_names()
print("Number of words:", len(words))
print("Document - words matrix:", word_count.shape)
print("First words:", words[0:100])

Number of documents: 2893
Number of words: 14279
Document - words matrix: (2893, 14279)
First words: ['aa', 'aal', 'aba', 'aback', 'abacus', 'abandon', 'abandoned', 'abandonment', 'abbas', 'abbreviation', 'abdomen', 'abduction', 'abed', 'aberrant', 'aberration', 'abide', 'abiding', 'abigail', 'ability', 'ablative', 'ablaut', 'able', 'abler', 'aboard', 'abolition', 'abord', 'aboriginal', 'aborigine', 'abound', 'abox', 'abreast', 'abridged', 'abroad', 'abrogate', 'abrook', 'abruptly', 'abscissa', 'absence', 'absent', 'absolute', 'absolutely', 'absoluteness', 'absolutist', 'absolutive', 'absolutization', 'absorbed', 'absorption', 'abstract', 'abstraction', 'abstractly', 'abstractness', 'absurd', 'absurdity', 'abu', 'abundance', 'abundant', 'abuse', 'abusive', 'abyss', 'academe', 'academic', 'academically', 'academician', 'academy', 'accelerate', 'accelerated', 'accelerative', 'accent', 'accentuate', 'accentuation', 'accept', 'acceptability', 'acceptable', 'acceptance', 'acceptation', 'acc

# Term frequency times inverse document frequency

After this first preprocessing, each document is summarized by a vector of size "number of words in the extracted dictionnary". For example, the first email in the list has become:

In [7]:
mail_number = 0
text = open(email_path[mail_number]).read()
print("Original email:")
print(text)
#print(LemmaTokenizer()(text))
#print(len(set(LemmaTokenizer()(text))))
#print(len([feat2word[i] for i in word_count2[mail_number, :].nonzero()[1]]))
#print(len([word_count2[mail_number, i] for i in word_count2[mail_number, :].nonzero()[1]]))
#print(set([feat2word[i] for i in word_count2[mail_number, :].nonzero()[1]])-set(LemmaTokenizer()(text)))
emailBagOfWords = {feat2word[i]: word_count[mail_number, i] for i in word_count[mail_number, :].nonzero()[1]}
print("Bag of words representation (", len(emailBagOfWords), " words in dict):", sep='')
print(emailBagOfWords)
print("\nVector reprensentation (", word_count[mail_number, :].nonzero()[1].shape[0], " non-zero elements):", sep='')
print(word_count[mail_number, :])

Original email:
Subject: new books : generative studies

we would like to bring to your attention to two new publications from john benjamins publishing in the field of generative studies : minimal words in a minimal syntax . word formation in swedish . gunlg josefsson 1998 ix , 199 pp . lingvistic aktuell / linguistics today , 19 us / canada : cloth : 1 55619 903 1 price : us $ 75 . 00 rest of the world : cloth : 90 272 2740 3 price : nlg 150 john benjamins publishing web site : http : / / www . benjamins . com for further information via e-mail : service @ benjamins . com in minimal words in a minimal syntax the author combines a detailed description of the morphological structure of words in swedish with a new approach to theoretical morphology based on the minimalist program of chomsky ( 1995 ) ( as developed for syntactic structure ) . the x - bar theoretic approach to word structure of the principles and parameters framework is replaced by a rule free approach incorporating only 

Counting words is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.

To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called `tf` for Term Frequencies.

Another refinement on top of `tf` is to downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus.

This downscaling is called `tf–idf` for “Term Frequency times Inverse Document Frequency” and again, scikit-learn does the job for us with the [TfidfTransformer](scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html) function.

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer().fit_transform(word_count)
tfidf.shape

(2893, 14279)

Now every email in the corpus has a vector representation that filters out unrelevant tokens and retains the significant information.

In [9]:
print("email 0:")
print(tfidf[0,:])

email 0:
  (0, 14188)	0.037094782848718
  (0, 14175)	0.0934722055784704
  (0, 14165)	0.04002254329922045
  (0, 14161)	0.14091834497600078
  (0, 14128)	0.048171529657830225
  (0, 14001)	0.04229702064162442
  (0, 13971)	0.13614332463320974
  (0, 13741)	0.10881502462586097
  (0, 13678)	0.07011630447737642
  (0, 13190)	0.1599996223643916
  (0, 13183)	0.07826614029245244
  (0, 13170)	0.039909654001890185
  (0, 12852)	0.11905309913954425
  (0, 12731)	0.09519381308974706
  (0, 12726)	0.05419821507889103
  (0, 12725)	0.08770551484025407
  (0, 12454)	0.0972311360436653
  (0, 12450)	0.11685136612701555
  (0, 12153)	0.017057510566715985
  (0, 12120)	0.05072806105260797
  (0, 12107)	0.3095911863729325
  (0, 11516)	0.15067900018468036
  (0, 11269)	0.12264747535784029
  (0, 10888)	0.07379085907896953
  (0, 10825)	0.15653228058490487
  :	:
  (0, 4988)	0.07936700679871678
  (0, 4983)	0.11982022298764217
  (0, 4742)	0.05338579912998475
  (0, 3380)	0.06934082955117414
  (0, 3341)	0.06172322925370016
  (

# Utility function

Let's put all this loading process into a separate file so that we can reuse it in other experiments.

In [10]:
import load_spam
spam_data = load_spam.spam_data_loader()
spam_data.load_data()

In [11]:
spam_data.print_email(8)

email file: ../data/lingspam_public/bare/part10/9-960msg1.txt
email is a spam: False
Subject: new series : studies in corpus linguistics

john benjamins publishing would like to call your attention to a new series ; studies in corpus linguistics studies in corpus linguistics aims to provide insights into the way a corpus can be used , the type of findings that can be obtained , the possible applications of these findings as well as the theoretical changes that corpus work can bring into linguistics and language engineering . the main concern of scl will be to present findings based on , or related to , the cumulative effect of naturally occurring language and on the interpretation of frequency and distributional data . general editor : elena tognini - bonelli consulting editor : wolfgang teubert terms in context jennifer pearson 1998 xii , 246 pp . studies in corpus linguistics , 1 us / canada : cloth : 1 55619 342 4 price : us $ 69 . 00 rest of the world : cloth : 90 272 2269 x price 